# 資訊工程師


# 軟體工程師


# 網頁設計師


In [1]:
import requests
import json
import re
import csv
from bs4 import BeautifulSoup
import time

job_url = 'https://www.1111.com.tw/job-bank/job-index.asp?si=1&ks={}&page={}'

DATA_PATH = '/tmp/hw_data'
filelist = []
words = []

# 抓出單頁


In [2]:
def get_list_page(url):
    
    HOST = 'https:'
    resp = requests.get(url)
    links = re.findall('<a href="(//www.1111.com.tw/job/.+/)" .+>.+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links    

In [104]:
!mkdir /tmp/hw_data

In [103]:
! rm -r -f /tmp/hw_data

In [6]:
DATA_PATH = '/tmp/hw_data'

def dump_page(url):  
    filename = url.split('/')[-2]
    resp = requests.get(url)
    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

In [7]:
def get_list(url):
    filename = url.split('/')[-2]
    filelist.append(filename)

In [74]:
def do_page(num1 ,a):   
    print('職缺 '+a)
    print('共抓'+str(num1)+'頁')
    count = 0
    filelist = []
    start = time.time()
    for page in range(num1) :
        the_url = job_url.format(a,page+1)
        for link in get_list_page(the_url):
            dump_page(link)
            count+=1
            get_list(link)
    end = time.time()
    elapsed = end - start
    print('共'+str(count)+'筆職缺')
    print("花費", elapsed, "秒")

In [91]:
do_page(50,"數據分析")

職缺 數據分析
共抓50頁
共977筆職缺
花費 184.90805864334106 秒


In [89]:
! ls /tmp/hw_data

In [92]:
filelist

['80092316',
 '80220571',
 '79921020',
 '80253675',
 '80135490',
 '79563481',
 '80245304',
 '78285538',
 '80182268',
 '79894887',
 '80037852',
 '77003665',
 '79872865',
 '78256232',
 '80255940',
 '80144017',
 '80173320',
 '80129159',
 '80257430',
 '80233458',
 '78403716',
 '80173626',
 '80211685',
 '80256610',
 '80117484',
 '80249539',
 '80078500',
 '80249607',
 '80249541',
 '80226025',
 '80170303',
 '79904042',
 '80211484',
 '80193412',
 '80049932',
 '78609801',
 '80237198',
 '80228087',
 '80155748',
 '80138636',
 '79948050',
 '80257519',
 '79002236',
 '80206164',
 '79105874',
 '79858559',
 '80193417',
 '79786061',
 '80106575',
 '80232218',
 '79846723',
 '79904129',
 '80100632',
 '80257426',
 '80192491',
 '79647021',
 '79748091',
 '80180527',
 '79937096',
 '79565745',
 '77115616',
 '55805134',
 '80171269',
 '80252635',
 '80254092',
 '79816850',
 '79875233',
 '77306780',
 '80147361',
 '80235988',
 '80237351',
 '80200104',
 '80240294',
 '80199141',
 '78518211',
 '77180438',
 '80037638',

# 爬內文

In [93]:
words 

[]

In [94]:
DATA_PATH = '/tmp/hw_data'
for l in filelist:
    with open(DATA_PATH + '/' + l) as f:
        html = f.read()
        soup = BeautifulSoup(html, 'lxml')
        data = soup.find_all('div',{'class':'listContent'})
        for a in range(len(data)):
            words.extend(data[a].text.split('、'))
filelist = []

In [95]:
with open('skill.txt') as sk:
    skilllist =  sk.read()
skill = skilllist.split('\n')
skill.remove('')

In [96]:
# 歸零
counter = {}
for b in skill:
    counter[b]=0

In [97]:
def get_skill(list):    
    for b in skill:
        counter[b] += 0
    for a in range(len(list)):
        word = list[a]
        if word in skill:                            
            counter[word]+=1
    return counter

In [98]:
dictskill = get_skill(words)

In [99]:
dictskill3 = sorted(dictskill.items(),key = lambda item:item[1],reverse=True)

# 輸出Json 、 CSV

In [65]:
def get_json(dict,file):
    with open(file, 'w') as f:
        json.dump(dict,f)

In [66]:
def get_csv(dict,file):
    with open(file,'w') as f:
        w=csv.writer(f)
        # write all keys on one row and all values on the next
        w.writerow(dict.keys())
        w.writerow(dict.values())

In [100]:
def get_tupletocsv(list,file):

    with open(file,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['技能','數量'])
        for row in list:
            csv_out.writerow(row)

In [101]:
get_tupletocsv(dictskill3,'數據分析.csv')

In [102]:
words = []